In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("raw_data/benchmark_results_mem.csv")
df.loc[df.forward_only, "Mode"] = "forward pass"
df.loc[~df.forward_only, "Mode"] = "training step"
df = df.rename(columns={"model.context_length": "Context"})[["Mode", "mixed_precision", "Context", "peak_memory"]]

In [ ]:
df

In [ ]:
mp = df[df.mixed_precision].pivot_table(index="Mode", columns="Context", values="peak_memory", observed=False)
fp = df[~df.mixed_precision].pivot_table(index="Mode", columns="Context", values="peak_memory", observed=False)

report = mp.combine(
    fp,
    lambda a, b: a.combine(
        b,
        lambda x, y: f"{x / (1024**3):.1f} / {y / (1024**3):.1f}"
    ),
)

In [ ]:
report

In [ ]:
def print_report(df, title):
    tex = df.to_latex(index=True, caption=f"{title} (bf16 / fp32)", escape=False)
    tex = tex.replace(r"\begin{table}", r"\begin{table}[H]")
    print(tex)

In [ ]:
print_report(report, "Peak memory in GB")